# Data preparation

Code authored by: Shaw Talebi

### imports

In [1]:
import pandas as pd
import numpy as np
from datasets import DatasetDict, Dataset

### load data

In [2]:
df = pd.read_json("dataset/paired_responses_v3.jsonl", lines=True)

In [3]:
df.head()

,pair_index,context,source,question,answer_a,answer_b,answer_b_preferred
0,0,\nKonteks 1:\nUU PPh Konsolidasi setelah UU 6 ...,UU UU PPh Konsolidasi setelah UU 6 Tahun 2023 ...,Apakah keuntungan dari penjualan saham oleh pe...,Keuntungan dari penjualan saham oleh pemegang ...,"Ya, keuntungan dari penjualan saham oleh pemeg...",1
1,1,\nKonteks 1:\nUU PPh Konsolidasi setelah UU 6 ...,UU UU PPh Konsolidasi setelah UU 6 Tahun 2023 ...,Apakah keuntungan dari penjualan saham oleh pe...,Keuntungan dari penjualan saham oleh pemegang ...,"Berdasarkan Pasal 18 Ayat 3c UU 6 Tahun 2023, ...",1
2,2,\nKonteks 1:\nUU PPh Konsolidasi setelah UU 6 ...,UU UU PPh Konsolidasi setelah UU 6 Tahun 2023 ...,Apakah keuntungan dari penjualan saham oleh pe...,Keuntungan dari penjualan saham oleh pemegang ...,"Berdasarkan Pasal 18 ayat 3c UU 6 Tahun 2023, ...",1
3,3,\nKonteks 1:\nUU PPh Konsolidasi setelah UU 6 ...,UU UU PPh Konsolidasi setelah UU 6 Tahun 2023 ...,Apakah keuntungan dari penjualan saham oleh pe...,Keuntungan dari penjualan saham oleh pemegang ...,"Berdasarkan Pasal 18 ayat 3c UU 6 Tahun 2023, ...",1
4,4,\nKonteks 1:\nUU PPh Konsolidasi setelah UU 6 ...,UU UU PPh Konsolidasi setelah UU 6 Tahun 2023 ...,Apakah keuntungan dari penjualan saham oleh pe...,"Ya, keuntungan dari penjualan saham oleh pemeg...","Berdasarkan Pasal 18 Ayat 3c UU 6 Tahun 2023, ...",1


### create prompt

In [4]:
def build_prompt(context, sumber, question):
    """
    Membuat struktur chat dengan role 'system' dan 'user'.
    System berisi konteks hukum + instruksi formal.
    User berisi pertanyaan.
    """
    SYSTEM_PROMPT = f"""
    Jawab pertanyaan berdasarkan konteks berikut:
    {context}

    Kamu adalah asisten ahli pajak Indonesia.
    Jawaban harus faktual, to the point, dan menggunakan bahasa formal.
    Jika informasi tidak ada atau pertanyaan tidak berkaitan dengan pajak,
    jawab: "Maaf, saya tidak memiliki pemahaman tentang hal itu."

    Sumber konteks: {sumber}

    Bila jawaban ditemukan dengan jelas di konteks:
    - Sertakan sumber pasal di akhir kalimat dengan cara yang natural,
    misalnya: "sesuai dengan Pasal ... UU Nomor ... Tahun ....".
    - Sertakan sumber hukum dengan format:
    Source: Pasal {{pasal}} Ayat {{ayat}} UU {{uu}}.

    Bila jawaban tidak ditemukan dengan jelas di konteks:
    Gunakan FORMAT JAWABAN AKHIR berikut:

    Sources Used:
    [Daftar sumber UU yang digunakan (minimal 2)]

    Summary:
    [Rangkuman inti analisis]

    PILIH SATU BAGIAN SAJA di bawah ini, lalu isi dengan teks yang relevan:

    [[ Conclusion ]]
    [Tulis kesimpulan, JIKA analisis berfokus pada ringkasan temuan
    dan implikasi logis dari data yang ada.]

    ATAU

    [[ Recommendation ]]Apa yang tidak boleh dilakukan penggugat sehubungan dengan pelaksanaan penagihan pajak?
    [Tulis rekomendasi, JIKA analisis berfokus pada usulan aksi,
    kebijakan, atau langkah perbaikan di masa depan.]
    """

    return [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": question}
    ]

In [5]:
df["prompt"] = df.apply(lambda x: build_prompt(x["context"], x["source"], x["question"]), axis=1)

### create chosen and rejected responses

In [6]:
def answer_to_chat(answer):
    return [{"role": "assistant", "content": answer.strip()}]

In [7]:
df["answer_a"] = df["answer_a"].apply(answer_to_chat)
df["answer_b"] = df["answer_b"].apply(answer_to_chat)

In [8]:
df["chosen"] = np.where(df["answer_b_preferred"] == 1, df["answer_b"], df["answer_a"])
df["rejected"] = np.where(df["answer_b_preferred"] == 1, df["answer_a"], df["answer_b"])

In [9]:
df_prep = df[["prompt", "chosen", "rejected"]]

In [10]:
df_prep.iloc[:,-3:].head()

,prompt,chosen,rejected
0,"[{'role': 'system', 'content': ' Jawab per...","[{'role': 'assistant', 'content': 'Ya, keuntun...","[{'role': 'assistant', 'content': 'Keuntungan ..."
1,"[{'role': 'system', 'content': ' Jawab per...","[{'role': 'assistant', 'content': 'Berdasarkan...","[{'role': 'assistant', 'content': 'Keuntungan ..."
2,"[{'role': 'system', 'content': ' Jawab per...","[{'role': 'assistant', 'content': 'Berdasarkan...","[{'role': 'assistant', 'content': 'Keuntungan ..."
3,"[{'role': 'system', 'content': ' Jawab per...","[{'role': 'assistant', 'content': 'Berdasarkan...","[{'role': 'assistant', 'content': 'Keuntungan ..."
4,"[{'role': 'system', 'content': ' Jawab per...","[{'role': 'assistant', 'content': 'Berdasarkan...","[{'role': 'assistant', 'content': 'Ya, keuntun..."


In [20]:
# write data to file
df_prep.to_csv('dataset/preferences.csv')

### Shuffle & split

In [11]:
df_shuffled = df_prep.sample(frac=1, random_state=42).reset_index(drop=True)

train_size = int(0.9 * len(df_shuffled))
df_train = df_shuffled.iloc[:train_size]
df_valid = df_shuffled.iloc[train_size:]

In [12]:
# Convert the pandas DataFrames back to Hugging Face Datasets
train_ds = Dataset.from_pandas(df_train)
valid_ds = Dataset.from_pandas(df_valid)

dataset_dict = DatasetDict({
    "train": train_ds,
    "valid": valid_ds
})

In [13]:
dataset_dict.save_to_disk("dataset/preferences_chatbot_hf_v2")
print("✅ DatasetDict saved to data/preferences_chatbot_hf/")

Saving the dataset (0/1 shards):   0%|          | 0/450 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/50 [00:00<?, ? examples/s]

✅ DatasetDict saved to data/preferences_chatbot_hf/


In [ ]:
df_prep.to_json("dataset/preferences_chatbot.jsonl", orient="records", lines=True, force_ascii=False)
print("✅ Data preference saved to data/preferences_chatbot.jsonl")

✅ Data preference saved to data/preferences_chatbot.jsonl


Saving the dataset (0/1 shards):   0%|          | 0/450 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/50 [00:00<?, ? examples/s]

✅ DatasetDict saved to data/preferences_chatbot_hf/


In [ ]:
dataset_dict.save_to_disk

DatasetDict({
    train: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 450
    })
    valid: Dataset({
        features: ['prompt', 'chosen', 'rejected'],
        num_rows: 50
    })
})